In [60]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import uuid
import time

Base = declarative_base()


class VehicleState(Base):
    """车辆状态数据表"""
    __tablename__ = 'vehicle_state'

    token = Column(String(36), primary_key=True)
    timestamp = Column(Float, nullable=False)
    x = Column(Float, nullable=False)
    y = Column(Float, nullable=False)
    heading = Column(Float, nullable=False)
    velocity = Column(Float, nullable=False)
    frame_id = Column(Integer, nullable=False)

    def __repr__(self):
        return f"VehicleState(frame={self.frame_id}, pos=({self.x:.2f}, {self.y:.2f}))"


class LidarData(Base):
    """激光雷达数据表"""
    __tablename__ = 'lidar'

    token = Column(String(36), primary_key=True)
    timestamp = Column(Float, nullable=False)
    filename = Column(String(255), nullable=False)
    num_points = Column(Integer, nullable=False)
    frame_id = Column(Integer, nullable=False)

    def __repr__(self):
        return f"LidarData(frame={self.frame_id}, points={self.num_points})"


def generate_frame_data(frame_id: int, base_time: float) -> tuple:
    """生成单帧数据"""
    frame_token = str(uuid.uuid4())
    timestamp = base_time + frame_id * 0.1  # 假设每帧间隔0.1秒

    vehicle_state = VehicleState(
        token=frame_token,
        frame_id=frame_id,
        timestamp=timestamp,
        x=100.0 + frame_id * 0.5,  # 每帧前进0.5米
        y=200.0,
        heading=1.57,
        velocity=5.0)

    lidar_data = LidarData(token=frame_token,
                           frame_id=frame_id,
                           timestamp=timestamp,
                           filename=f'lidar/frame_{frame_id:03d}.pcd',
                           num_points=100000)

    return vehicle_state, lidar_data


def create_multi_frame_dataset(num_frames: int = 30):
    """创建多帧数据集"""
    # 创建数据库
    db_dir = Path("./")
    db_file = db_dir / "autodrive_multi_frame_scene.db"

    # 如果数据库文件已存在，先删除它
    if db_file.exists():
        db_file.unlink()

    # 创建数据库引擎
    engine = create_engine(f'sqlite:///{db_file.absolute()}')

    # 创建表
    Base.metadata.create_all(engine)

    # 创建会话
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # 生成基准时间
        base_time = time.time()

        # 逐帧生成和保存数据
        for frame_id in range(num_frames):
            vehicle_state, lidar = generate_frame_data(frame_id, base_time)
            session.add(vehicle_state)
            session.add(lidar)

        # 提交事务
        session.commit()
        print(f"Successfully inserted {num_frames} frames of data")

        # 验证插入的数据
        vehicle_count = session.query(VehicleState).count()
        lidar_count = session.query(LidarData).count()
        print(
            f"Verified data: {vehicle_count} vehicle states, {lidar_count} lidar frames"
        )

    except Exception as e:
        session.rollback()
        print(f"Error occurred: {e}")
        raise
    finally:
        session.close()


def verify_database():
    """验证数据库内容"""
    db_file = Path("./autodrive_multi_frame_scene.db")
    engine = create_engine(f'sqlite:///{db_file.absolute()}')
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # 查询并显示部分数据
        print("\nSample vehicle states:")
        vehicles = session.query(VehicleState).limit(5).all()
        for v in vehicles:
            print(
                f"Frame {v.frame_id}: pos=({v.x:.2f}, {v.y:.2f}), v={v.velocity:.2f}"
            )

        print("\nSample lidar data:")
        lidars = session.query(LidarData).limit(5).all()
        for l in lidars:
            print(f"Frame {l.frame_id}: {l.filename}, points={l.num_points}")

    finally:
        session.close()


if __name__ == "__main__":
    create_multi_frame_dataset(30)
    verify_database()

Successfully inserted 30 frames of data
Verified data: 30 vehicle states, 30 lidar frames

Sample vehicle states:
Frame 0: pos=(100.00, 200.00), v=5.00
Frame 1: pos=(100.50, 200.00), v=5.00
Frame 2: pos=(101.00, 200.00), v=5.00
Frame 3: pos=(101.50, 200.00), v=5.00
Frame 4: pos=(102.00, 200.00), v=5.00

Sample lidar data:
Frame 0: lidar/frame_000.pcd, points=100000
Frame 1: lidar/frame_001.pcd, points=100000
Frame 2: lidar/frame_002.pcd, points=100000
Frame 3: lidar/frame_003.pcd, points=100000
Frame 4: lidar/frame_004.pcd, points=100000
